## MLOps with Azure ML Pipelines

ML Pipeline - Training & Registration.  ML Pipelines can help you to build, optimize and manage your machine learning workflow. 

ML Pipelines encapsulate a workflow for a machine learning task.  Tasks often include:
- Data Prep
- Training 
- Publishing Models
- Deployment of Models

First we will set some key variables to be leveraged inside the notebook

In [ ]:
subscription_id = os.getenv("SUBSCRIPTION_ID", default="")
resource_group = os.getenv("RESOURCE_GROUP", default="")
workspace_name = os.getenv("WORKSPACE_NAME", default="")
workspace_region = os.getenv("WORKSPACE_REGION", default="")
cluster_name = os.getenv("CLUSTER_NAME", default="")
pipeline_name = os.getenv("PIPELINE_NAME", default="")
model_name = os.getenv("MODEL_NAME", default="")
build_id = os.getenv("BUILD_BUILDID", default='1')

print('subscription_id = ' + str(subscription_id))
print('resource_group = ' + str(resource_group))
print('workspace_name = ' + str(workspace_name))
print('workspace_region = ' + str(workspace_region))
print('cluster_name = ' + str(cluster_name))
print('pipeline_name = ' + str(pipeline_name))
print('model_name = ' + str(model_name))
print('build_id=' + str(build_id))
print('build_number = ' + build_number)

In [1]:
registered_env_name = "experiment_env"
experiment_folder = 'exp_train_pipeline'
dataset_prefix_name = 'exp'
cluster_name = "mm-cluster"

Import required packages

In [2]:
# Import required packages
from azureml.core import Workspace, Experiment, Datastore, Environment, Dataset
from azureml.core.compute import ComputeTarget, AmlCompute, DataFactoryCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_CPU_IMAGE
from azureml.pipeline.core import Pipeline, PipelineParameter, PipelineData
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import PipelineParameter, PipelineData
from azureml.data.output_dataset_config import OutputTabularDatasetConfig, OutputDatasetConfig, OutputFileDatasetConfig
from azureml.data.datapath import DataPath
from azureml.data.data_reference import DataReference
from azureml.data.sql_data_reference import SqlDataReference
from azureml.pipeline.steps import DataTransferStep
import logging

### Connect to the workspace and create a cluster for running the AML Pipeline

Connect to the AML workspace and the default datastore. To run an AML Pipeline, we will want to create compute if a compute cluster is not already available

In [3]:
# Connect to AML Workspace
#ws = Workspace.from_config()

try:
    ws = Workspace(subscription_id=subscription_id, 
                   resource_group=resource_group, 
                   workspace_name=workspace_name)
    print("Workspace configuration succeeded. Skip the workspace creation steps below")
except:
    print("Workspace does not exist. Creating workspace")
    ws = Workspace.create(name=workspace_name, subscription_id=subscription_id, resource_group=resource_group,
                            location=workspace_region, create_resource_group=True, sku='enterprise', exist_ok=True)
    

# Get the default datastore
default_ds = ws.get_default_datastore()

#Select AML Compute Cluster
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException


try:
    # Check for existing compute target
    pipeline_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        pipeline_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        pipeline_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

Found existing cluster, use it.


## Create Run configuration

The RunConfiguration defines the environment used across all the python steps.  There are a variety of ways of setting up an environment.  An environment holds the required python packages needed for your code to execute on a compute cluster

In [4]:
import os
# Create a folder for the pipeline step files
os.makedirs(experiment_folder, exist_ok=True)

print(experiment_folder)

exp_train_pipeline


In [5]:
#conda_yml_file = '../configuration/environment.yml'
conda_yml_file = './'+ experiment_folder+ '/environment.yml'

In [6]:
%%writefile $conda_yml_file
name: experiment_env
dependencies:
- python=3.6.2
- scikit-learn
- ipykernel
- matplotlib
- pandas
- pip
- pip:
  - azureml-defaults
  - pyarrow
  - azureml-monitoring
  - azureml-interpret
  - inference-schema
  - joblib
  - azure-ml-api-sdk

Overwriting ./exp_train_pipeline/environment.yml


In [7]:
# Create a Python environment for the experiment (from a .yml file)
env = Environment.from_conda_specification("experiment_env", conda_yml_file)


run_config = RunConfiguration()
run_config.docker.use_docker = True
run_config.environment = env
run_config.environment.docker.base_image = DEFAULT_CPU_IMAGE

In [8]:
registered_env_name

'experiment_env'

In [9]:
from azureml.core import Environment
from azureml.core.runconfig import RunConfiguration

# Create a Python environment for the experiment (from a .yml file)
experiment_env = Environment.from_conda_specification(registered_env_name, conda_yml_file)

# Register the environment 
experiment_env.register(workspace=ws)
registered_env = Environment.get(ws, registered_env_name)

# Create a new runconfig object for the pipeline
pipeline_run_config = RunConfiguration()

# Use the compute you created above. 
pipeline_run_config.target = pipeline_cluster

# Assign the environment to the run configuration
pipeline_run_config.environment = registered_env

print ("Run configuration created.")

Run configuration created.


## Define Output datasets


The **OutputFileDatasetConfig** object is a special kind of data reference that is used for interim storage locations that can be passed between pipeline steps, so you'll create one and use at as the output for the first step and the input for the second step. Note that you need to pass it as a script argument so your code can access the datastore location referenced by the data reference. 

Note, in all cases we specify the datastore that should hold the datasets and whether they should be registered following step completion or not. This can optionally be disabled by removing the register_on_complete() call.

These can be viewed in the Datasets tab directly in the AML Portal

In [10]:
#get data from storage location and save to exp_raw_data
exp_raw_data       = OutputFileDatasetConfig(name='Exp_Raw_Data', destination=(default_ds, dataset_prefix_name + '_raw_data/{run-id}')).read_delimited_files().register_on_complete(name= dataset_prefix_name + '_Raw_Data')

#data split into testing and training
exp_training_data  = OutputFileDatasetConfig(name='Exp_Training_Data', destination=(default_ds, dataset_prefix_name + '_training_data/{run-id}')).read_delimited_files().register_on_complete(name=dataset_prefix_name + '_Training_Data')
exp_testing_data   = OutputFileDatasetConfig(name='Exp_Testing_Data', destination=(default_ds, dataset_prefix_name + '_testing_data/{run-id}')).read_delimited_files().register_on_complete(name=dataset_prefix_name + '_Testing_Data')

## Define Pipeline Data

Data used in pipeline can be **produced by one step** and **consumed in another step** by providing a PipelineData object as an output of one step and an input of one or more subsequent steps

This can be leveraged for moving a model from one step into another for model evaluation

### Create Python Script Step

In [11]:
get_data_step = PythonScriptStep(
    name='Get Data',
    script_name='get_data.py',
    arguments =['--exp_raw_data', exp_raw_data],
    outputs=[exp_raw_data],
    compute_target=pipeline_cluster,
    source_directory='./' + experiment_folder,
    allow_reuse=False,
    runconfig=pipeline_run_config
)

In [12]:
%%writefile ./$experiment_folder/get_data.py

from azureml.core import Run, Workspace, Datastore, Dataset
from azureml.data.datapath import DataPath
import pandas as pd
import os
import argparse
from sklearn import preprocessing
import numpy as np

#Parse input arguments
#command-line parsing module 
parser = argparse.ArgumentParser("Get data from and register in AML workspace")
parser.add_argument('--exp_raw_data', dest='exp_raw_data', required=True)

args, _ = parser.parse_known_args()
exp_raw_dataset = args.exp_raw_data

#Get current run
current_run = Run.get_context()

#Get associated AML workspace
ws = current_run.experiment.workspace

#Connect to default data store
ds = ws.get_default_datastore()

tab_data_set = Dataset.Tabular.from_delimited_files(path=(ds, 'diabetes-data/*.csv'))

raw_df = tab_data_set.to_pandas_dataframe()

#Make directory on mounted storage
os.makedirs(exp_raw_dataset, exist_ok=True)

#this will allow us to register the dataset on completion
raw_df.to_csv(os.path.join(exp_raw_dataset, 'exp_raw_data.csv'), index=False)

Overwriting ./exp_train_pipeline/get_data.py


### Split Data Step

In [13]:
split_scale_step = PythonScriptStep(
    name='Split  Raw Data',
    script_name='split.py',
    arguments =['--exp_training_data', exp_training_data,
                '--exp_testing_data', exp_testing_data],
    inputs=[exp_raw_data.as_input(name='Exp_Raw_Data')],
    outputs=[exp_training_data, exp_testing_data],
    compute_target=pipeline_cluster,
    source_directory='./' + experiment_folder,
    allow_reuse=False,
    runconfig=pipeline_run_config
)

In [14]:
%%writefile ./$experiment_folder/split.py

from azureml.core import Run, Workspace, Datastore, Dataset
from azureml.data.datapath import DataPath
import os
import argparse

import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import joblib
from numpy.random import seed

#Parse input arguments
parser = argparse.ArgumentParser("Split raw data into train/test and scale appropriately")

parser.add_argument('--exp_training_data', dest='exp_training_data', required=True)
parser.add_argument('--exp_testing_data', dest='exp_testing_data', required=True)


args, _ = parser.parse_known_args()
exp_training_data = args.exp_training_data
exp_testing_data = args.exp_testing_data


#Get current run
current_run = Run.get_context()

#Get associated AML workspace
ws = current_run.experiment.workspace

# Read input dataset to pandas dataframe
raw_datset = current_run.input_datasets['Exp_Raw_Data']
raw_df = raw_datset.to_pandas_dataframe()


for col in raw_df.columns:
    missing = raw_df[col].isnull()
    num_missing = np.sum(missing)
    if num_missing > 0:  
        raw_df['quality_{}_ismissing'.format(col)] = missing


print(raw_df.columns)

#Split data into training set and test set
df_train, df_test = train_test_split(raw_df, test_size=0.3, random_state=0)





# Save train data to both train and test (reflects the usage pattern in this sample. Note: test/train sets are typically distinct data).
os.makedirs(exp_training_data, exist_ok=True)
os.makedirs(exp_testing_data, exist_ok=True)

df_train.to_csv(os.path.join(exp_training_data, 'exp_training_data.csv'), index=False)
df_test.to_csv(os.path.join(exp_testing_data, 'exp_testing_data.csv'), index=False)


Overwriting ./exp_train_pipeline/split.py


In [15]:
### TrainingStep

In [16]:
#Raw data will be preprocessed and registered as train/test datasets

model_file = PipelineData(name='model_file', datastore=default_ds)

#by specifying as input, it does not need to be included in the arguments
train_model_step = PythonScriptStep(
    name='Train',
    script_name='train.py',
    arguments =['--model_file_output', model_file],
    inputs=[
            exp_training_data.as_input(name='Exp_Training_Data'),
            exp_testing_data.as_input(name='Exp_Testing_Data'),
           ],
    outputs = [model_file],
    compute_target=pipeline_cluster,
    source_directory='./' + experiment_folder,
    allow_reuse=False,
    runconfig=pipeline_run_config
)


In [76]:
%%writefile ./$experiment_folder/train.py

from azureml.core import Run, Workspace, Datastore, Dataset
from azureml.data.datapath import DataPath
import os
import argparse
import shutil

import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve


import matplotlib.pyplot as plt
import joblib
from numpy.random import seed


#Parse input arguments
parser = argparse.ArgumentParser("Train Logistic Regression model")
parser.add_argument('--model_file_output', dest='model_file_output', required=True)


args, _ = parser.parse_known_args()
model_file_output = args.model_file_output

def converttypes(df):
    cols = df.columns
    for c in cols:
        df[c] = pd.to_numeric(df[c], errors = 'coerce')

    print('data types')
    print(df.dtypes)
    return df


#Get current run
run = Run.get_context()

#Get associated AML workspace
ws = run.experiment.workspace

# Read input dataset to pandas dataframe
X_train_dataset = run.input_datasets['Exp_Training_Data'].to_pandas_dataframe()
X_test_dataset = run.input_datasets['Exp_Testing_Data'].to_pandas_dataframe()

X_train_dataset = converttypes(X_train_dataset)
X_test_dataset = converttypes(X_test_dataset)


print(type(X_train_dataset))

# Separate features and labels
X_train, y_train = X_train_dataset[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, X_train_dataset['Diabetic'].values
X_test, y_test   = X_test_dataset[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, X_test_dataset['Diabetic'].values

print('***********')
print(type(X_train))
print(type(X_test))
print(type(y_train[0]))
print(type(y_test[0]))
print('**********')
# Set regularization hyperparameter
reg = 0.01

# Train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

print('y_hat[0] is if type=:' + str(type(y_hat[0])))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

run.parent.log(name='AUC', value=np.float(auc))
run.parent.log(name='Accuracy', value=np.float(acc))

# Save the trained model in the outputs folder
os.makedirs('./outputs', exist_ok=True)
joblib.dump(value=model, filename='./outputs/diabetes_model_remote.pkl')

os.makedirs(model_file_output, exist_ok=True)

shutil.copyfile('./outputs/diabetes_model_remote.pkl', os.path.join(model_file_output, 'diabetes_model_remote.pkl'))


Overwriting ./exp_train_pipeline/train.py


### Evaluate Model Step

In [77]:
#Evaluate and register model here
#Compare metrics from current model and register if better than current
#best model


deploy_file = PipelineData(name='deploy_file', datastore=default_ds)

evaluate_and_register_step = PythonScriptStep(
    name='Evaluate and Register Model',
    script_name='evaluate_and_register.py',
    arguments=[
        '--model_file', model_file,
        '--deploy_file_output', deploy_file,       
    ],
    inputs=[model_file.as_input('model_file'),
            exp_training_data.as_input(name='Exp_Training_Data'),
            exp_testing_data.as_input(name='Exp_Testing_Data')
           ],
    outputs=[ deploy_file],
    compute_target=pipeline_cluster,
    source_directory='./' + experiment_folder,
    allow_reuse=False,
    runconfig=pipeline_run_config
)

In [78]:
%%writefile ./$experiment_folder/evaluate_and_register.py

from azureml.core import Run, Workspace, Datastore, Dataset
from azureml.core.model import Model
from azureml.data.datapath import DataPath

import joblib
import os
import argparse
import shutil
import pandas as pd

from interpret.ext.blackbox import TabularExplainer
from azureml.interpret import ExplanationClient
from azureml.interpret.scoring.scoring_explainer import LinearScoringExplainer, save

from azureml.core.model import InferenceConfig
from azureml.core.compute import ComputeTarget, AksCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.webservice import Webservice, AksWebservice


parser = argparse.ArgumentParser("Evaluate model and register if more performant")

parser.add_argument('--model_file', type=str, required=True)
parser.add_argument('--deploy_file_output', type=str, help='File passing in pipeline to deploy')

args, _ = parser.parse_known_args()

deploy_file = args.deploy_file_output
model_file = args.model_file

def converttypes(df):
    cols = df.columns
    for c in cols:
        df[c] = pd.to_numeric(df[c], errors = 'coerce')

    print('data types')
    print(df.dtypes)
    return df

def model_explain():
    #load trinning data
    X_train_dataset = run.input_datasets['Exp_Training_Data'].to_pandas_dataframe()
    X_test_dataset = run.input_datasets['Exp_Testing_Data'].to_pandas_dataframe()
    
    X_test_dataset = converttypes(X_test_dataset)
    X_train_dataset = converttypes(X_train_dataset)
    
    X_train, y_train = X_train_dataset[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, X_train_dataset['Diabetic'].values
    X_test, y_test   = X_test_dataset[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, X_test_dataset['Diabetic'].values

    
    #load the model
    model_list = Model.list(ws, name=model_name, latest=True)
    model_path = model_list[0].download(exist_ok=True)
    model = joblib.load(model_path)

    #https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/explain-model/azure-integration/scoring-time/train_explain.py
    # create an explanation client to store the explanation (contrib API)
    client = ExplanationClient.from_run(run)

    # create an explainer to validate or debug the model
    tabular_explainer = TabularExplainer(model,
                                         initialization_examples=X_train,
                                         features=['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age'],
                                         classes=[0, 1])
                                         #transformations=transformations)

    # explain overall model predictions (global explanation)
    # passing in test dataset for evaluation examples - note it must be a representative sample of the original data
    # more data (e.g. x_train) will likely lead to higher accuracy, but at a time cost
    global_explanation = tabular_explainer.explain_global(X_test)

    # uploading model explanation data for storage or visualization
    comment = 'Global explanation on classification model trained'
    client.upload_model_explanation(global_explanation, comment=comment, model_id=model_reg.id)



#Get current run
run = Run.get_context()

#Get associated AML workspace
ws = run.experiment.workspace

#Get default datastore
ds = ws.get_default_datastore()


#Get metrics associated with current parent run
metrics = run.get_metrics()

print('current run metrics')
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')


print('parent run metrics')
#Get metrics associated with current parent run
metrics = run.parent.get_metrics()

for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')

current_model_AUC = float(metrics['AUC'])
current_model_accuracy = float(metrics['Accuracy'])

# Get current model from workspace
model_name = 'diabetes_model_remote'
model_description = 'Diabetes model remote'
model_list = Model.list(ws, name=model_name, latest=True)
first_registration = len(model_list)==0

updated_tags = {'AUC': current_model_AUC}

print('updated tags')
print(updated_tags)

# Copy  training outputs to relative path for registration



relative_model_path = 'outputs'
run.upload_folder(name=relative_model_path, path=model_file)



#If no model exists register the current model
if first_registration:
    print('First model registration.')
    model_reg = run.register_model(model_path='outputs/diabetes_model_remote.pkl', model_name=model_name,
                   tags=updated_tags,
                   properties={'AUC': current_model_AUC})

    #model_explain()
else:
    #If a model has been registered previously, check to see if current model 
    #performs better. If so, register it.
    print(dir(model_list[0]))
    if float(model_list[0].tags['AUC']) < current_model_AUC:
        print('New model performs better than existing model. Register it.')

        model_reg = run.register_model(model_path='outputs/diabetes_model_remote.pkl', model_name=model_name,
                   tags=updated_tags,
                   properties={'AUC': current_model_AUC, 'Accuracy': current_model_accuracy})

        #model_explain()
        
        # Output accuracy to file
        with open(deploy_file, 'w+') as f:
            f.write(('deploy'))
    
    else:
        print('New model does not perform better than existing model. Cancel run.')
        
        with open(deploy_file, 'w+') as f:
            f.write(('no deployment'))
            
        run.cancel()

Overwriting ./exp_train_pipeline/evaluate_and_register.py


### Deploy ACI

In [79]:
# exp_deploy_pipeline_data = PipelineData(
#         name='scoring_url_file', 
#         pipeline_output_name='scoring_url_file',
#         datastore=default_ds,
#         output_mode='mount',
#         is_directory=False)

scoring_file = PipelineData(name='scoring_file', datastore=default_ds)

aci_service_name = 'diabetes-service-remote-training'
registered_model_name = 'diabetes_model_remote'

env_name = PipelineParameter(name='environment_name', default_value=registered_env_name)
service_name = PipelineParameter(name='service_name', default_value=aci_service_name)
model_name = PipelineParameter(name='model_name', default_value=registered_model_name)



deploy_test = PythonScriptStep(
    name='Deploy to ACI',
    script_name='deployACI.py',
    arguments=[
        '--scoring_file_output', scoring_file,
        '--deploy_file', deploy_file,
        '--environment_name', env_name,
        '--service_name', service_name,
        '--model_name', model_name
        
    ],
    inputs=[
        deploy_file.as_input('deploy_file'),
            
    ],
    outputs=[scoring_file],
    compute_target=pipeline_cluster,
    source_directory='./' + experiment_folder,
    allow_reuse=False,
    runconfig=pipeline_run_config
)

In [80]:
%%writefile ./$experiment_folder/deployACI.py

import argparse
from azureml.core import Workspace, Environment
from azureml.core.model import Model
from azureml.core.run import Run
from azureml.core.model import InferenceConfig
from azureml.core.webservice import Webservice, AciWebservice
from azureml.exceptions import WebserviceException

parser = argparse.ArgumentParser(description='Deploy arg parser')
parser.add_argument('--scoring_file_output', type=str, help='File storing the scoring url')
parser.add_argument('--deploy_file', type=str, help='File storing if model should be deployed')
parser.add_argument('--environment_name', type=str,help='Environment name')
parser.add_argument('--service_name', type=str,help='service name')
parser.add_argument('--model_name', type=str,help='model name')



args = parser.parse_args()
scoring_url_file = args.scoring_file_output
deploy_file      = args.deploy_file
environment_name = args.environment_name
service_name     = args.service_name
model_name       = args.model_name


run = Run.get_context()

#Get associated AML workspace
ws = run.experiment.workspace

model = Model(ws, model_name)
env = Environment.get(ws, environment_name)
inference_config = InferenceConfig(entry_script='score.py', environment=env)

# Deploy model
aci_config = AciWebservice.deploy_configuration(
            cpu_cores = 1, 
            memory_gb = 2, 
            tags = {'model': 'diabetes remote training'},
            auth_enabled=True,
            enable_app_insights=True,
            collect_model_data=True)

try:
    service = Webservice(ws, name=service_name)
    if service:
        service.delete()
except WebserviceException as e:
         print()

service = Model.deploy(ws, service_name, [model], inference_config, aci_config)
service.wait_for_deployment(True)
    

# Output scoring url
print(service.scoring_uri)
with open(scoring_url_file, 'w+') as f:
    f.write(service.scoring_uri)



Overwriting ./exp_train_pipeline/deployACI.py


In [81]:
%%writefile ./$experiment_folder/score.py

import json
import joblib
import numpy as np
from azureml.core.model import Model
from azureml.monitoring import ModelDataCollector
import time
import os
import pandas as pd


#version 2
# Called when the service is loaded
def init():
    global model
    #Print statement for appinsights custom traces:
    print ("model initialized" + time.strftime("%H:%M:%S"))
    # Get the path to the deployed model file and load it
    path = os.path.join(Model.get_model_path('diabetes_model_remote'))
    
    print(path)
    model = joblib.load(path)

    
    global inputs_dc, prediction_dc
    inputs_dc = ModelDataCollector("best_model", designation="inputs", feature_names=['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age'])
    prediction_dc = ModelDataCollector("best_model", designation="predictions", feature_names=["Diabetic"])



# Called when a request is received
def run(raw_data):
    try:
        # Get the input data as a numpy array
        #data = np.array(json.loads(raw_data)['data'])
        # Get a prediction from the model
        
        json_data = json.loads(raw_data)
        predictions = model.predict(json_data['data'])
        print ("Prediction created" + time.strftime("%H:%M:%S"))
        # Get the corresponding classname for each prediction (0 or 1)
        classnames = ['not-diabetic', 'diabetic']
        predicted_classes = []
        for prediction in predictions:
            val = int(prediction)
            predicted_classes.append(classnames[val])
        # Return the predictions as JSON
        
         # Log the input and output data to appinsights:
        info = {
            "input": raw_data,
            "output": predicted_classes
            }
        print(json.dumps(info))
        
        inputs_dc.collect(json_data['data']) #this call is saving our input data into Azure Blob
        prediction_dc.collect(predicted_classes) #this call is saving our prediction data into Azure Blob

        
        return json.dumps(predicted_classes)
    except Exception as e:
        error = str(e)
        print (error + time.strftime("%H:%M:%S"))
        return error

Overwriting ./exp_train_pipeline/score.py


## Create Pipeline steps

## Create Pipeline
Create an Azure ML Pipeline by specifying the steps to be executed. Note: based on the dataset dependencies between steps, exection occurs logically such that no step will execute unless all of the necessary input datasets have been generated.

In [82]:
pipeline = Pipeline(workspace=ws, steps=[get_data_step, split_scale_step, train_model_step, evaluate_and_register_step, deploy_test])

In [86]:
experiment = Experiment(ws, 'AML_Automation_RemotePipelineTraining')
run = experiment.submit(pipeline)
#run.wait_for_completion(show_output=True)

Submitted PipelineRun 18ab20d5-6ff5-4421-8512-38894ae7ea0b
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/18ab20d5-6ff5-4421-8512-38894ae7ea0b?wsid=/subscriptions/5da07161-3770-4a4b-aa43-418cbbb627cf/resourcegroups/mm-aml-dev-ops-rg/workspaces/mm-aml-dev-ops&tid=72f988bf-86f1-41af-91ab-2d7cd011db47


## Testing with downloaded Model

In [85]:
from azureml.core.model import Model
import joblib
import pandas as pd
import json

model_list = Model.list(ws, name='diabetes_model_remote', latest=True)
#model_list = Model.list(ws, name='diabetes_model', latest=True)
print(model_list)
model_path = model_list[0].download(exist_ok=True)
model = joblib.load(model_path)

#run for downloaded model
x_new = [[2,180,74,24,21,23.9091702,1.488172308,22]]
print ('Patient: {}'.format(x_new[0]))

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

json_data = json.loads(input_json)
results = model.predict(json_data['data'])
print((results))

classnames = ['not-diabetic', 'diabetic']
predicted_classes = []
for prediction in results:
    val = int(prediction)
    predicted_classes.append(classnames[val])

print(predicted_classes)

[Model(workspace=Workspace.create(name='mm-aml-dev-ops', subscription_id='5da07161-3770-4a4b-aa43-418cbbb627cf', resource_group='mm-aml-dev-ops-rg'), name=diabetes_model_remote, id=diabetes_model_remote:12, version=12, tags={'AUC': '0.8484934573859395'}, properties={'AUC': '0.8484934573859395', 'Accuracy': '0.774'})]
Patient: [2, 180, 74, 24, 21, 23.9091702, 1.488172308, 22]
['0']
['not-diabetic']


/anaconda/envs/azureml_py36/lib/python3.6/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.24.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


## Testing with Deployed Model

In [74]:
??Webservice

In [75]:
import json
import requests
from azureml.core.webservice import Webservice

def MakePrediction():
    endpoint_url = url
    x_new = [[2,180,74,24,21,23.9091702,1.488172308,22]]
    input_json = json.dumps({"data": x_new})
    print(input_json)
    body = input_json
    r = requests.post(endpoint_url, headers=headers, data=body)
    return (r.json())

#get service endpoint
service = Webservice(workspace=ws, name='diabetes-service-remote-training')
print(service.state)
if service.state == 'Healthy':
    url = service.scoring_uri
    print(url)
    api_key = service.get_keys()[0]
    print(api_key)
    headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}
    results = MakePrediction()
    print(results)
else:
    print('Service must be in health state to make API call')





Healthy
http://79b57900-3ccf-4574-856e-27a162e85a03.eastus.azurecontainer.io/score
1gzYOEwJN9qfhajdUbYbuxdg3y9lfQhQ
{"data": [[2, 180, 74, 24, 21, 23.9091702, 1.488172308, 22]]}
'list' object has no attribute 'tolist'


## Publish Pipeline

In [87]:
published_pipeline = pipeline.publish(name = 'Diabetes Training Pipeline',
                                     description = 'Pipeline that generates batch predictions using a registered trained model.',
                                     continue_on_step_failure = False)

In [ ]:
#published_pipeline

In [ ]:
# from azureml.pipeline.core import ScheduleRecurrence, Schedule

# # Submit the Pipeline every Monday at 00:00 UTC
# recurrence = ScheduleRecurrence(frequency="Week", interval=1, week_days=["Monday"], time_of_day="00:00")
# weekly_schedule = Schedule.create(ws, name="weekly-diabetes-training", 
#                                   description="Based on time",
#                                   pipeline_id=published_pipeline.id, 
#                                   experiment_name='mslearn-diabetes-pipeline', 
#                                   recurrence=recurrence)
# print('Pipeline scheduled.')